In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_canal"
TABELA_DESTINO = "v_credit.gold.dm_canal"

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_canal"),
    F.col("nm_canal"),
    F.col("st_ativo"),
    
    F.when(
        (F.upper(F.col("nm_canal")).like("%URA%")) | 
        (F.upper(F.col("nm_canal")).like("%CHAT%")) | 
        (F.upper(F.col("nm_canal")).like("%BOT%")), 
        "Automatizado"
    ).otherwise("Humanizado").alias("ds_tipo_interacao"),
    
    F.current_timestamp().alias("dt_ingestion"),
    F.lit("ETL_Databricks_Merge").alias("dc_origem")
)

deltaTable = DeltaTable.forName(spark, TABELA_DESTINO)

deltaTable.alias("target").merge(
    df_gold.alias("source"),
    "target.cd_canal = source.cd_canal"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Merge realizado com sucesso em {TABELA_DESTINO}")